In [54]:
input_string = '(C*a+a)'

In [55]:
import json

In [56]:
first = dict()
first['exp'] = ['a', 'C', '(']
first['arith_exp'] = ['a', 'C', '(']
first['arith_exp_dash'] = ['+', '-', 'e']
first['term'] = ['a', 'C', '(']
first['term_dash'] = ['*', '/', 'e']
first['factor'] = ['a', 'C', '(']
first['ident'] = ['a']
first['const'] = ['C']
first['rel'] = ['<', '=', '>']
first['plus'] = ['+', '-']
first['mul'] = ['*', '/']
first['operator'] = ['a']

In [57]:
def get_first(nonterm):
    global i
    if i > len(input_string) - 1:
        return False
    is_nonterm = input_string[i] in first[nonterm]
    return is_nonterm

def rel():
    global tree
    tree += '{"Rel":['
    global i
    if input_string[i] == '<':
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
        if input_string[i] == '=':
            tree += ','
            tree += '"'
            tree += input_string[i]
            tree += '"'
            i += 1
        elif input_string[i] == '>':
            tree += ','
            tree += '"'
            tree += input_string[i]
            tree += '"'
            i += 1
    elif input_string[i] == '=':
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
    elif input_string[i] == '>':
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
        if input_string[i] == '=':
            tree += ','
            tree += '"'
            tree += input_string[i]
            tree += '"'
            i += 1
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'

def const():
    global tree
    tree += '{"Const":['
    global i
    if get_first('const'):
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'
    
def ident():
    global tree
    tree += '{"Id":['
    global i
    if get_first('ident'):
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'

def mul():
    global tree
    tree += '{"Mul":['
    global i
    if get_first('mul'):
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'

def plus():
    global tree
    tree += '{"Plus":['
    global i
    if get_first('plus'):
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'

def factor():
    global tree
    tree += '{"Factor":['
    global i
    if get_first('ident'):
        ident()
    elif get_first('const'):
        const()
    elif input_string[i] == '(':
        tree += '"(",'
        i += 1
        exp()
        tree += ','
        if input_string[i] != ')':
            raise Exception('Mismatching brackets')
        else:
            i += 1
            tree += '")"'
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'

def term_dash():
    global tree
    tree += '{"Term_Dash":['
    if get_first('mul'):
        mul()
        tree += ','
        factor()
        tree += ','
        term_dash()
    tree += ']}'

def term():
    global tree
    tree += '{"Term":['
    factor()
    tree += ','
    term_dash()
    tree += ']}'

def arith_exp_dash():
    global tree
    tree += '{"Arithmetic_Expression_Dash":['
    if get_first('plus'):
        plus()
        tree += ','
        term()
        tree += ','
        arith_exp_dash()
    tree += ']}'

def arith_exp():
    global tree
    tree += '{"Arithmetic_Expression":['
    term()
    tree += ','
    arith_exp_dash()
    tree += ']}'

def exp():
    global tree
    tree += '{"Expression":['
    if get_first('arith_exp'):
        arith_exp()
        if get_first('rel'):
            tree += ','
            rel()
            tree += ','
            arith_exp() 
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'   

def parse():
    global i 
    global tree

    i = 0
    tree = ''

    try:
        exp()
        if i < len(input_string):
           raise Exception('Unexpected symbol on ' + str(i))
        print('Ok!')
        return True
    except Exception as e: 
        print(e)
        return False

i = 0
tree = ''
result = dict()
if parse():
    result = json.loads(tree)

Ok!


In [58]:
result

{'Expression': [{'Arithmetic_Expression': [{'Term': [{'Factor': ['(',
        {'Expression': [{'Arithmetic_Expression': [{'Term': [{'Factor': [{'Const': ['C']}]},
              {'Term_Dash': [{'Mul': ['*']},
                {'Factor': [{'Id': ['a']}]},
                {'Term_Dash': []}]}]},
            {'Arithmetic_Expression_Dash': [{'Plus': ['+']},
              {'Term': [{'Factor': [{'Id': ['a']}]}, {'Term_Dash': []}]},
              {'Arithmetic_Expression_Dash': []}]}]}]},
        ')']},
      {'Term_Dash': []}]},
    {'Arithmetic_Expression_Dash': []}]}]}

In [59]:
input_string = "{a:=(C*a+a);a:=C;}"

In [60]:
def operator():
    global tree
    tree += '{"Operator":['
    global i
    ident()
    tree += ','
    if input_string[i] == ':':
        tree += '"'
        tree += input_string[i]
        tree += '",'
        i += 1
        if input_string[i] == '=':
            tree += '"'
            tree += input_string[i]
            tree += '",'
            i += 1
        else:
            raise Exception('Unexpected symbol on ' + str(i))
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    exp()
    tree += ']}'

def tail():
    global tree
    tree += '{"Tail":['
    global i
    if input_string[i] == ';':
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
        if get_first('operator'):
            tree += ','
            operator()
            tree += ','
            tail()
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'

def oper_list():
    global tree
    tree += '{"Operator_List":['
    operator()
    tree += ','
    tail()
    tree += ']}'

def block():
    global tree
    tree += '{"Block":['
    global i 
    if input_string[i] == '{':
        tree += '"'
        tree += input_string[i]
        tree += '",'
        i += 1
        oper_list()
        tree += ','
        if input_string[i] == '}':
            tree += '"'
            tree += input_string[i]
            tree += '"'
            i += 1
        else:
            raise Exception('Unmatched { brackets or unexpected symbol')
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'

def prog():
    global tree
    tree += '{"Program":['
    block()
    tree += ']}'

def prog_parse():
    global i 
    i = 0  
    global tree
    tree = '' 
  
    try:
       prog()
       if i < len(input_string) - 1:
           raise Exception('Unexpected symbol on ' + str(i))
       print('Ok!')
       return True
    except Exception as e: 
       print(e)
       return False

i = 0
tree = ''
result = dict()
if prog_parse():
    result = json.loads(tree)


Ok!


In [53]:
result

{'Program': [{'Block': ['{',
    {'Operator_List': [{'Operator': [{'Id': ['a']},
        ':',
        '=',
        {'Expression': [{'Arithmetic_Expression': [{'Term': [{'Factor': ['(',
                {'Expression': [{'Arithmetic_Expression': [{'Term': [{'Factor': [{'Const': ['C']}]},
                      {'Term_Dash': [{'Mul': ['*']},
                        {'Factor': [{'Id': ['a']}]},
                        {'Term_Dash': []}]}]},
                    {'Arithmetic_Expression_Dash': [{'Plus': ['+']},
                      {'Term': [{'Factor': [{'Id': ['a']}]},
                        {'Term_Dash': []}]},
                      {'Arithmetic_Expression_Dash': []}]}]}]},
                ')']},
              {'Term_Dash': []}]},
            {'Arithmetic_Expression_Dash': []}]}]}]},
      {'Tail': [';',
        {'Operator': [{'Id': ['a']},
          ':',
          '=',
          {'Expression': [{'Arithmetic_Expression': [{'Term': [{'Factor': [{'Const': ['C']}]},
                {'Term_Dash